Benson = EGUB/03658

51.6201, -1.0983

In [ ]:
import datetime as dt
import glob
import os

import cfgrib
import numpy as np
import requests
import xarray as xr
from dask.distributed import wait, Client as DaskClient
# from ecmwf.opendata import Client as ECMWFClient

In [ ]:
dask_client = DaskClient(n_workers=5, threads_per_worker=2)

## Dynamical - GFS

In [ ]:
ds_gfs = xr.open_zarr("https://data.dynamical.org/noaa/gfs/forecast/latest.zarr?email=email@email.com")
ds_gfs

In [ ]:
# variable_list = [
#     "pressure_reduced_to_mean_sea_level",
#     "relative_humidity_2m", 
#     "temperature_2m", 
#     "total_cloud_cover_atmosphere", 
#     "wind_u_10m",
#     "wind_v_10m"
#     ]

variable_list = [
    "relative_humidity_2m", 
    "temperature_2m", 
    "total_cloud_cover_atmosphere", 
    "wind_u_10m",
    "wind_v_10m",
]

ds_gfs_subset = ds_gfs[variable_list].sel(
    latitude=slice(52, 51.25),
    longitude=slice(-1.5, -0.75),
    lead_time=slice(np.timedelta64(0, 'h'), np.timedelta64(119, 'h')),
    init_time=slice("2024-10-24T00:00:00.000000000", "2025-10-24T18:00:00.000000000"),
    )
ds_gfs_subset

In [ ]:
ds_gfs_subset.load()
ds_gfs_subset.to_zarr("data/gfs_benson_1yr.zarr", mode="a")

ds_gfs.close()
ds_gfs_subset.close()

In [ ]:
ds = xr.open_zarr("/Users/Shared/scratch/ml_weather_forecasting/gfs_benson_1yr.zarr")
ds

In [ ]:
# Occassionally the download may miss some data, we can check for nan values and then return dates with missing data to re-download
missing_dates_list = set(np.where(np.isnan(ds.wind_v_10m))[0].tolist())

missing_dates_list

## ECMWF - IFS

Data starts 2023-01-18

There have been some changes to ECMWF's file structures over the last couple of years - these are all of the combinations I've found so far. There are also dates where AIFS is missing, as they were experimenting with it.

- url1 to 20240131 00z = [yyyymmdd]/[run]z/0p4-beta/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2

`url = f"{url_base}/{date_str}/{run}z/0p4-beta/oper/{date_str}{run}0000-{step}h-oper-fc.grib2"`

- url2 to 20240228 00z = [yyyymmdd]/[run]z/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2
- url3 to 20250123 18z = [yyyymmdd]/[run]z/[ifs|aifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2
- url4 to 20250209 06z = [yyyymmdd]/[run]z/[ifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2    * missing AIFS *
- url5 to 20250225 00z = [yyyymmdd]/[run]z/[ifs|aifs-single]/0p25/[experimental]/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2     * Experimental for AIFS *
- url6 to 20250227 18z =[yyyymmdd]/[run]z/[ifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2    * missing AIFS *
- url7 to now = [yyyymmdd]/[run]z/[ifs|aifs-single]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2

`url = f"{url_base}/{date_str}/{run}z/aifs-single/0p25/oper/{date_str}{run}0000-{step}h-oper-fc.grib2"`

In [ ]:
date_str = "20241024"
run = "00"
step = "0"
base_url = "https://storage.googleapis.com/ecmwf-open-data"
response = requests.get(f"{base_url}/{date_str}/{run}z/ifs/0p25/oper/{date_str}{run}0000-{step}h-oper-fc.grib2")

grib_file = "test_grib.grib2"
if response.status_code == 200:
    with open(grib_file, "wb") as f:
        f.write(response.content)
else:
    print(f"Couldn't download {date_str} {run}z {step}h - Status code: {response.status_code}")

ds_list = cfgrib.open_datasets(grib_file)
ds_list_final = []
vars = ['t2m', 'd2m', 'msl', 'u10', 'v10', 'tcc']
for ds_n in ds_list:
    var_names = [var for var in vars if var in ds_n.data_vars]
    if len(var_names) > 0:
        if 'heightAboveGround' in ds_n.coords:
            ds_n = ds_n.drop_vars('heightAboveGround')
        ds_list_final.append(ds_n[var_names])

ds = xr.merge(ds_list_final, compat="no_conflicts")
ds

In [ ]:
def request_url_process_data(url, grib_file_path, nc_file_path, date_str, run, step):
    response = requests.get(url)

    if response.status_code == 200:
        with open(grib_file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {date_str} {run}z {step}h")
    else:
        print(f"Couldn't download {date_str} {run}z {step}h - Status code: {response.status_code}")

    ds_list = cfgrib.open_datasets(grib_file_path)
    ds_list_final = []
    vars = ['t2m', 'd2m', 'msl', 'u10', 'v10', 'tcc']
    for ds_n in ds_list:
        var_names = [var for var in vars if var in ds_n.data_vars]
        if len(var_names) > 0:
            if 'heightAboveGround' in ds_n.coords:
                ds_n = ds_n.drop_vars('heightAboveGround')
            ds_list_final.append(ds_n[var_names])

    ds = xr.merge(ds_list_final, compat="no_conflicts")
    ds = ds.sel(latitude=slice(52, 51.25), longitude=slice(-1.5, -0.75))
    ds.to_netcdf(nc_file_path)
    print(f"Saved {nc_file_path}")

    ds.close()
    for f in glob.glob(f"{grib_file_path}*"):
        os.remove(f)

In [ ]:
def url_picker(url_base: str, model: str, date: dt.datetime, run: str, step: int) -> str|None:
    date_str = date.strftime("%Y%m%d")

    if model == "aifs":
        model = "aifs-single"

    url_dict = {
        "url1":f"{url_base}/{date_str}/{run}z/0p4-beta/oper/{date_str}{run}0000-{step}h-oper-fc.grib2",
        "url2":f"{url_base}/{date_str}/{run}z/0p25/oper/{date_str}{run}0000-{step}h-oper-fc.grib2",
        "url3":f"{url_base}/{date_str}/{run}z/{model}/0p25/oper/{date_str}{run}0000-{step}h-oper-fc.grib2",
        }
    
    if dt.datetime(2023, 1, 18) <= date <= dt.datetime(2024, 1, 31):
        if date == dt.datetime(2024, 1, 31) and (run in ["06", "12", "18"]):
            return url_dict["url2"]
        else:
            return url_dict["url1"]
    elif dt.datetime(2024, 2, 1) <= date <= dt.datetime(2024, 2, 28):
        if date == dt.datetime(2024, 2, 28) and (run in ["06", "12", "18"]):
            return url_dict["url3"]
        else:
            return url_dict["url2"]
    elif dt.datetime(2024, 3, 1) <= date:
        return url_dict["url3"]

In [ ]:
# url_base = "https://storage.googleapis.com/ecmwf-open-data"
url_base = "https://ecmwf-forecasts.s3.eu-central-1.amazonaws.com"
model = "ifs"

# Create a list of dates to be downloaded
start_dt = dt.datetime(2024, 10, 24)
end_dt = dt.datetime(2025, 10, 24)

if end_dt > dt.datetime.now():
    print("End date is in the future, setting to today's date")
    end_dt = dt.datetime.now()
if model == "aifs" and start_dt < dt.datetime(2025, 2, 28):
    raise Exception("AIFS data not available before 2025-02-28")

delta = dt.timedelta(days=1)

dates = []
while start_dt <= end_dt:
    dates.append(start_dt)
    start_dt += delta

# Choose what runs to be downloaded
runs = ["00", "12"]

# Choose what steps to be downloaded
# IFS:
# - For times 00z &12z: 0 to 144 by 3, 150 to 360 by 6.
# - For times 06z & 18z: 0 to 144 by 3.
#
# AIFS:
# - 6 hourly steps to 360 (15 days)

if model == "ifs":
    steps = list(range(0, 73, 3))
elif model == "aifs":
    steps = list(range(0, 121, 6))

# Setup loops over dates, runs, steps
futures = []
ds_list = []
for date in dates:
    date_str = date.strftime("%Y%m%d")

    for run in runs:
        date_folder = f"data/ecmwf_{model}/{date_str}_{run}z"
        os.makedirs(date_folder, exist_ok=True)

        for step in steps:
            # Create URL based on the date to be downloaded, using the formats found in the data stores
            url = url_picker(url_base, model, date, run, step)

            grib_file_path = os.path.join(date_folder, f"{model}_{date_str}_{run}z_{step}h.grib2")
            nc_file_path = os.path.join(date_folder, f"{model}_{date_str}_{run}z_{step}h.nc")

            future = dask_client.submit(request_url_process_data, url, grib_file_path, nc_file_path, date_str, run, step)
            futures.append(future)

wait(futures)